<a href="https://colab.research.google.com/github/HUJameson/Colab/blob/main/aillm_0210.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#不要执行，非常耗token

import os,openai,backoff
import pandas as pd

openai.api_key = os.getenv("OPENAI_API_KEY")
dynasties= ['唐', '宋', '元', '明', '清', '汉', '魏', '晋', '南北朝']
super_powers = ['隐形', '飞行', '读心术', '瞬间移动', '不死之身', '喷火']
story_types = ['轻松', '努力', '艰难']

@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def gpt35(prompt, max_tokens=2048, temperature=0.5, top_p=1, frequency_penalty=0, presence_penalty=0):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        frequency_penalty=frequency_penalty,
        presence_penalty=presence_penalty)
    return response["choices"][0]["text"]

def prepare_stories(dynasties, super_powers, story_types, output_file="data/ultraman_stories.csv"):
    df = pd.DataFrame()
    repeat = 3
    for dynasty in dynasties:
        for super_power in super_powers:
            for story_type in story_types:
                   for i in range(repeat):
                        prompt = f"""请你用中文写一段300字的故事，情节跌宕起伏，讲述一位{dynasty}朝时期的英雄人物，穿越到现代，拥有了{super_power}这样的超能力，通过{story_type}的战斗，帮助奥特曼一起打败了怪兽的故事。"""
                        story = gpt35(prompt)
                        row = {"dynasty": dynasty, "super_power": super_power, "story_type": story_type, "story": story}
                        row = pd.DataFrame([row])
                        df = pd.concat([df, row], axis=0, ignore_index=True)

    df.to_csv(output_file)

prepare_stories(dynasties, super_powers, story_types)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

Mounted at /content/drive


In [ ]:
!pip install openai
import openai
from sk_utils import read_sk
openai_sk = read_sk()
%env OPENAI_API_KEY=$openai_sk

In [ ]:
import pandas as pd

df = pd.read_csv("data/ultraman_stories.csv")
df['sub_prompt'] = df['dynasty'] + "," + df['super_power'] + "," + df['story_type']
prepared_data = df.loc[:,['sub_prompt','story']]
prepared_data.rename(columns={'sub_prompt':'prompt', 'story':'completion'}, inplace=True)
prepared_data.to_csv('data/prepared_data.csv',index=False)

import subprocess

subprocess.run('openai tools fine_tunes.prepare_data --file data/prepared_data.csv --quiet'.split())

In [17]:
output = subprocess.run('openai api fine_tunes.create --training_file data/prepared_data_prepared.jsonl --model curie --suffix "ultraman"'.split(), stdout=subprocess.PIPE).stdout
print(output)

b"Found potentially duplicated files with name 'prepared_data_prepared.jsonl', purpose 'fine-tune' and size 446199 bytes\nfile-eoIlel2JuBwCBGq8EaYoL44f\nfile-lujkljdVWAzs9dHFYqoIh4UY\nEnter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: Uploaded file from data/prepared_data_prepared.jsonl: file-KMS7n1GydW73gGtD58uW6Sfl\nCreated fine-tune: ft-gUum6Ej7gYJghTfFcmdwkeHJ\nStreaming events until fine-tuning is complete...\n\n(Ctrl-C will interrupt the stream, but not cancel the fine-tune)\n[2023-10-05 02:43:17] Created fine-tune: ft-gUum6Ej7gYJghTfFcmdwkeHJ\n[2023-10-05 02:43:42] Fine-tune failed. Fine-tune will exceed free trial credits\n\nJob failed. Please contact us through our help center at help.openai.com if you need assistance.\n"


In [14]:
output = subprocess.run('openai api fine_tunes.list'.split(), stdout=subprocess.PIPE).stdout
print(output)

b'{\n  "object": "list",\n  "data": [\n    {\n      "object": "fine-tune",\n      "id": "ft-DUHvHf64kCgg4NZXCIoFx03Y",\n      "hyperparams": {\n        "n_epochs": 4,\n        "batch_size": null,\n        "prompt_loss_weight": 0.01,\n        "learning_rate_multiplier": null\n      },\n      "organization_id": "org-LsRSFcCdezFMWrF1yYEd6eYe",\n      "model": "curie",\n      "training_files": [\n        {\n          "object": "file",\n          "id": "file-eoIlel2JuBwCBGq8EaYoL44f",\n          "purpose": "fine-tune",\n          "filename": "data/prepared_data_prepared.jsonl",\n          "bytes": 446199,\n          "created_at": 1696472353,\n          "status": "processed",\n          "status_details": null\n        }\n      ],\n      "validation_files": [],\n      "result_files": [],\n      "created_at": 1696472353,\n      "updated_at": 1696472377,\n      "status": "failed",\n      "fine_tuned_model": null\n    },\n    {\n      "object": "fine-tune",\n      "id": "ft-XQtqlfiK0vt629xbTKohD

In [19]:

output = subprocess.run('openai api fine_tunes.results -i ft-gUum6Ej7gYJghTfFcmdwkeHJ'.split(), stdout=subprocess.PIPE).stdout
print(output)

b''


In [15]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def write_a_story(prompt):
    response = openai.Completion.create(
        model="curie:ft-bothub-ai:ultraman-2023-04-04-03-03-26",
        prompt=prompt,
        temperature=0.7,
        max_tokens=2000,
        top_p=1,
        stop=["."])
    return response["choices"][0]["text"]

story = write_a_story("宋,发射激光,艰难 ->\n")
print(story)

InvalidRequestError: ignored